### Il est explicité ici l'utilisation des ensembles de codes écrits pour l'analyse de sensibilité sur des champs stochastiques et variables aléatoires.

### Il est d'abord conseillé d'installer l'environnement virtuel dont la définition se trouvent dans le fichier yaml

Nous allons faire ici l'analyse de sensiblité sur une poutre en flexion représentée par 100 éléments finis, et ou le module young et le diamètre de chaque élément est déterminé par un processus gaussien en une dimension. La position de la force, sa norme, tout comme la densité du matérieau sont déterminés par des lois nomales gaussiennes.
Dans la logique d'écriture de ces codes, il faut avoir un à priori sur les processus gaussiens, la loi qu'ils suivent, tout comme sur les paramètres des lois gaussiennes. Ensuite, il faut avoir une fonction python qui prend en entrée ces champs, et qui renvoie un ensemble connu de resultas. 


In [ ]:
try:
    import anastruct, openturns, numba, joblib
except:
    import os
    if os.sys.platform == 'linux' :
        file_path = 'sensitivityEnv.yml'
        os.system('conda env create -f'+file_path)
        # to have the right modules installed
        print('now activate the environment and restart jupyter with other kernel')
    else :
        print('Do it alone')

In [ ]:
import sys
sys.path.insert(0, '../')
# Voici les deux scripts destinés à gérer l'analyse de sensibilité sur les champs stochastiques
import StochasticProcessSensitivity as SPS
import StochasticProcessConstructor as SPC
# Classes utilitaires
import numpy                        as np
import openturns                    as ot
import matplotlib.pyplot            as plt
from   importlib                import reload 
# on importe aussi les fonctions à étudier
import RandomBeamGenerationClass    as rbgc

D'abord, nous définisson l'ensemble de nos variables d'entrée, tout comme les variables de sortie :
- L'on va définir un par un tout les processus et variables aléatoires utilisées dans notre model. Bien sûr cela implique d'avoir un à-priori sur le comportement de ces différentes lois probabilistes. 
- Néanmoins, comme nous en sommes à des codes d'essai, il serait assez trivial de rajouter la prossiblité de récuperer l'approximation d'un champ inconnu avec l'approcimation de Karhunen - Loeve, en présence d'un grand nombre de mesures.
- La définition des éléments sur lesquels est construit le champ stochastique est un peu différent de la manière interne à openturns. En effet, s'étant placés directement dans un cadre de poutre en 'éléments finis, ou l'on a N+1 noeuds (N étant le nombre de poutres), il faut savoir si N est le nombre de mailles ou le nombre de noeuds. Dans le choix a été fait ici de définir la taille du maillage (grid_shape) de la manière suivante :
> grid_shape = [[position_X0, longeur_totaleX, nombre_mailles], [position_Y0, longeur_totaleY, nombre_mailles], ..]
 

In [ ]:
# process governing the young modulus for each element (MPa)
process_E = SPC.StochasticProcessConstructor(dimension=1,
                                        grid_shape=[[0,1000,100],],
                                        covariance_model={'Model':'MaternModel',
                                                        'amplitude':[50000],
                                                        'scale':[300],
                                                        'nu':13/3},
                                        trend_arguments=['x'],trend_function=210000)
process_E.setName('E_')


# process governing the diameter for each element (mm) 
# other constructor shown 
process_D = SPC.StochasticProcessConstructor()
process_D.setDimension(1)
process_D.setGrid([[0,1000,100],])
#setCovarianceModel also accepts any of the models defined in openturns, already initialised
#with the right parameters, not only dictionaries
process_D.setCovarianceModel(covarianceModelDict = {'Model':'MaternModel',
                                                    'amplitude':[.3],
                                                    'scale':[250],
                                                    'nu':7.4/3})
process_D.setTrend(['X'], 10)
process_D.setName('D_')

# random variable for the density of the material (kg/m³)
rho         = 7850.
sigma       = 750
nameD       = 'Rho'
RV_Rho = ot.Normal(rho, sigma)
RV_Rho.setName(nameD)

# random variable for the position of the force   (mm) 
middle       = 500
sigma_f      = 50
namePos     = 'FP'
RV_Fpos = ot.Normal(middle, sigma_f)
RV_Fpos.setName(namePos)

# random variable for the norm of the force    (N)
muForce       = 100
# we go from sigma = 15 to sigma = 1.5, as the influence is too important
sigma_Fnor    = 5.5
nameNor       = 'FN'
RV_Fnorm  = ot.Normal(muForce, sigma_Fnor)
RV_Fnorm.setName(nameNor)

Il est important de noter que les processus et variables aléatoires doivent être définies à partir du module 
NdGaussianProcessConstructor, qui contient les classes suivantes : 
###### Classe pour construire un champ stochastique : 
> NdGaussianProcessConstructor.NdGaussianProcessConstructor() 
###### Classe pour construire une variables aléatoire  : 
> NdGaussianProcessConstructor.NormalDistribution() ## Classe openturns.Normal() suchargée
###### Classe pour construire un vecteur de variables aléatoires normales : 
> NdGaussianProcessConstructor.RandomNormalVector() ## Classe openturns.PythonRandomVector surchargée. Cette dernière est utilisée en interne par NdGaussianProcessConstructor.

**Finalement, l'autre particularité du NdGaussianProcessConstructor, est d'utiliser un objet _numpy.memmap_ modifiée, qui enregistre les échantillons de processus gaussiens sous forme de fichier temporaire dans le directoire d'utilisation des codes et les éfface en sortant du code**

L'on définit ensuite les variables de sortie. Il faut connaître l'ordre dans lequel la fonction renvoie ses resultats connaitre leur nom comme leur dimension. 
    

In [ ]:
from importlib import reload
reload(SPS)
outputVariables = {'out1' :
                   {
                         'name'     : 'VonMisesStress',
                         'position' : 0,
                         'shape'    : (102,)  
                    },
                   'out2' :
                   {
                        'name'      : 'maxDeflection',
                        'position'  : 1,
                        'shape'     : (1,)
                   }
                  }
#Pour utiliser notre fonction, un wrapper a été spécialement écrit pour faciliter l'accès aux fonctions mais
#ce choix est entièrement dépendant de la manière dont a été définie la fonction sur laquelle vous travaillez.
functionWrapper = rbgc.sampleAndSoloFunctionWrapper(process_E, process_D, RV_Rho, RV_Fpos, RV_Fnorm)

###### Ensuite, on crée une instance pour l'analyse de sensibilité.
Il faut donner en entrée une liste contenant les procéssus et les variables aléatoires **dans l'ordre** dans lequel la fonction les recoit. 

*Pour cela il faut aussi connaître l'ordre des variables d'entrée de la fonction et leur nombre.*

**Ces fonctions sont celles qui prennent entrée des champs aléatoires (vecteurs et matrices, de type *list ou numpy*) et variables aléatoire (scalaires (vecteurs 1D pour le multiprocessing)) et non pas les fonctions prennant en entrée les variables aléatoires issues de la décomposition de la Karhunen Loeve, qui elles sont construites en interne dans la classe**

In [ ]:
inputVarList = [process_E, process_D, RV_Rho, RV_Fpos, RV_Fnorm]
# We also the need the two functions of the model (one for samples, the other for single evaluations)
# In our case, as our model is defined as a class, we have to first create the model, 
# but it also could just be just two functions taking as an input the fields and RVs
soloFunction   = functionWrapper.randomBeamFunctionSolo
sampleFunction = functionWrapper.randomBeamFunctionSample
##
size           = 50 ## Number of samples for our sobol indicies experiment (kept low here to make things faster)
##
reload(SPS)
## It is sufficient just to pass one of the functions 
processSensitivityAnalysis = SPS.StochasticProcessSensitivityAnalysis(inputVarList, 
                                                                       outputVariables,
                                                                       sampleFunction,
                                                                       soloFunction,
                                                                       size)

Voici une vue du dictionnaire intermédiaire qui est crée une fois les processus et variables aléatoires d'entrées définies :

On voit que la position des processus dans les arguments de la fonction d'entrée est enregistrée dans le dictionnaire dans la clé *position* et viennent de la manière de laquelle on a mis les VA et Processus dans le vecteur inputVarList. 


###### Ensuite, grace à l'intermédiaire de la classe ot.SobolIndiciesExperiment, on génère les' variables aléatoires d'entrée 

In [ ]:
processSensitivityAnalysis.run(generationType = 1)


###### Ensuite on récupère les sorties associées aux entrées génerées. 
L'idée du postprocessing est d'identifier si dans les sorties il y a des valeurs de type np.nan, et de refaire les experiences manquantes tant qu'il y a des np.nan de présents.

**Ceci crée quelques problèmes : En effet, si le calcul numérique n'a pas pu aboutir avec une certaine réalisation des variables en entrée, est-ce que cela veut dire que cette réalisation est défaillante? Va-t-on l'inclure dans le cas du calcul de la défaillance ou de la sensibilité? Car si on l'inclut pas, il se peut qu'on oublie un nombre conséquent de modes défaillants. Solution : changer le modèle informatique et le rendre robuste à ces erreurs, ou faire une étude précise avec ces réalisation particulières.**


La solution retenue pour génerer les experiences manquantes est d'itérer au dessus de chaque index de la sortie ou se trouvent les nans, de refaire une experience de sobol de taille 1 ( donc qui renverra d+2 valeurs, avec d la dimension de l'entrée), de générer les sorties correspondant aux entrées (recommencer si il s'y trouve un np.nan), et de remplacer les d+2 valeurs du inputDesign d'entrée avec celles que l'on vient de regénérer. 

Le *timing* vient de la classe ***custum_wraps***, qui est pour l'instant assez inutile, mais permet de prendre en main les décorateurs...

Une fois le _output design_ generé, l'on peut faire l'analyse de sensibilité sur le model.
Pour ce faire, l'on utilise la fonction ***self.getSobolIndiciesKLCoefs***.
La fonction ne renvoie pas directement les indices de sobol, mais bien des champs d'objets ***openTURNS.SaltelliSensitivityAlgorithm*** ou ***openTURNS.MartinezSensitivityAlgorithm*** ou ***openTURNS.JansenSensitivityAlgorithm*** ou ***openTURNS.MauntzKucherenkoSensitivityAlgorithm***.

Ceci est bien sûr fait dans l'optique de toujours pouvoir accéder à l'entierté des indices de sobol et indices totaux du modèle.
Ceci veut aussi dire qu'il manque une dimension pour les champs en sortie, celle des variables d'entrée.

In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (13, 5)
import StochasticProcessSensitivityIndices as ngpsi
_=reload(ngpsi)

Ici on calcule les indices de sobol du premier ordre

In [ ]:
output = processSensitivityAnalysis.outputDesignList; output1 = output[0]; output2 = output[1]

In [9]:
sensitivity=ngpsi.SobolIndicesStochasticProcessAlgorithm(output1,50)

Implicit dimension = 7
Implicit description: [X0,X1,X2,X3,X4,X5,X6]


In [10]:
sensitivity.getFirstOrderIndices()

There are 5 indices to get in (102,) dimensions with 102 elements
basic output shape is: (50, 102)
output reshaped into matrix of shape (dim<=2)  [50, 102]
data for variance calculus prepared 
 X_fo shape is (50, 102) Y_fo shape is (50, 102) 

Variance ND is: [0.00958383 0.00828492 0.0087516  0.00939705 0.01018164 0.01110133
 0.01214819 0.01330526 0.01454586 0.01583497 0.01713223 0.01839598
 0.01958775 0.02067597 0.02163834 0.02246225 0.02314345 0.02368362
 0.02408732 0.02435911 0.02450109 0.02451111 0.0243819  0.02410127
 0.02365369 0.02302325 0.02219791 0.02117433 0.01996215 0.01858632
 0.01708666 0.01551436 0.01392664 0.01238088 0.01092973 0.00961774
 0.0084792  0.00753687 0.00680102 0.00626904 0.00587629 0.00573705
 0.00541826 0.00548224 0.00661553 0.00842481 0.00973597 0.01144594
 0.01303367 0.01188672 0.00898187 0.00974971 0.01459836 0.01940236
 0.01884757 0.01698237 0.01604845 0.01567713 0.01555022 0.01571382
 0.01638599 0.01664959 0.01552473 0.01436397 0.01452232 0.01456154
 0.

Variance ND is: [0.03588734 0.05501753 0.05743024 0.05888254 0.06009133 0.06111961
 0.06197076 0.06265424 0.06319726 0.06364267 0.06404211 0.06444899
 0.06491364 0.0654797  0.0661796  0.06702662 0.06800393 0.06905349
 0.07007038 0.070907   0.0713899  0.07134729 0.07064236 0.06920404
 0.06704703 0.06427453 0.06106161 0.05762297 0.05417399 0.05089633
 0.04791759 0.04530835 0.04309384 0.04127255 0.03983428 0.03877272
 0.03809111 0.03780216 0.03792417 0.03847555 0.0394176  0.04042207
 0.04234964 0.04532435 0.05229791 0.06612711 0.08016404 0.09728328
 0.11569859 0.12918397 0.15127589 0.15181458 0.15455106 0.14280837
 0.1250959  0.10953924 0.09427729 0.08162997 0.072238   0.06591395
 0.06192092 0.05777324 0.05660619 0.05737445 0.05886577 0.06008992
 0.06101517 0.061682   0.06219997 0.06274028 0.06351725 0.06476149
 0.06668789 0.06946235 0.07317067 0.07779402 0.08319518 0.08911978
 0.09521473 0.10106323 0.10623173 0.11032075 0.11301055 0.11409339
 0.11348803 0.11123663 0.1074877  0.10247065 0

array([[-7.06658054e-02, -1.20999695e-01, -1.14384589e-01,
        -1.04816340e-01, -9.42653026e-02, -8.31290298e-02,
        -7.16692583e-02, -6.01458756e-02, -4.88318320e-02,
        -3.80033412e-02, -2.79203359e-02, -1.88022383e-02,
        -1.08041789e-02, -3.99996540e-03,  1.62258517e-03,
         6.15159526e-03,  9.72855519e-03,  1.25189927e-02,
         1.46830774e-02,  1.63512531e-02,  1.76072178e-02,
         1.84783250e-02,  1.89325678e-02,  1.88817225e-02,
         1.81912787e-02,  1.66985531e-02,  1.42400027e-02,
         1.06867066e-02,  5.98348312e-03,  1.83424109e-04,
        -6.53213564e-03, -1.38561914e-02, -2.13832718e-02,
        -2.86558654e-02, -3.52190537e-02, -4.06710568e-02,
        -4.47003963e-02, -4.71055852e-02, -4.77982194e-02,
        -4.67951587e-02, -4.76660669e-02, -4.19377980e-02,
        -2.65715427e-02, -9.70190024e-03,  1.12189263e-02,
         4.19761742e-02,  5.46669405e-02,  5.99079285e-02,
         7.70639915e-02,  7.36959360e-02,  6.40087821e-0

In [11]:
S, Stot, varS, varStot = ngpsi.SobolIndicesStochasticProcessAlgorithm.getFirs(output1,1000)
S2, Stot2, varS2, varS2tot = ngpsi.SobolIndicesStochasticProcessAlgorithm.getSobolIndices(output2,1000)

AttributeError: module 'StochasticProcessSensitivityIndices' has no attribute 'NdGaussianProcessSensitivityIndicesBase'

L'on peut récuperer ces indices du premier ordre dans ***self.processSensitivityAnalysis.sensitivityResults.firstOrderIndices***
Matplotlib permet de facilement visualiser ces indices. Néanmoins, cette visualisation est pour l'instant peu pertinente, puisque l'on visualise chaque variables aléatoire de Karhunen Loeve individuellement, alors qu'il nous faut encore combiner ces dernières pour que l'indice de sobol soit représentatif du champ stochastique consideré.

In [ ]:
varNames = ["Module d'Young",'Diametre','Densité','Position Force','Norme Force']
ngpsi.plotSobolIndicesWithErr(S2, varS2, varNames, 5, Stot2, varS2tot)

In [ ]:
ngpsi.plotSobolIndicesWithErr(S,varS, varNames, 5)

In [ ]:
ngpsi.plotSobolIndicesWithErr(Stot,varStot, varNames, 5)

In [ ]:
plt.style.use('default')
plt.yticks([0,1,2,3,4],["Module d'Young",'Diametre','Densité','Position Force','Norme Force'])
x=plt.imshow(varS, cmap='CMRmap',aspect='auto', interpolation= None)
plt.colorbar()

In [ ]:
plt.yticks([0,1,2,3,4],["Module d'Young",'Diametre','Densité','Position Force','Norme Force'])
x=plt.imshow(S, cmap='CMRmap',aspect='auto')
plt.colorbar()

In [ ]:
plt.yticks([0,1,2,3,4],["Module d'Young",'Diametre','Densité','Position Force','Norme Force'])
x=plt.imshow(Stot, cmap='CMRmap',aspect='auto')
plt.colorbar()

In [ ]:
plt.yticks([0,1,2,3,4],["Module d'Young",'Diametre','Densité','Position Force','Norme Force'])
x=plt.imshow(varStot, cmap='CMRmap',aspect='auto')
plt.colorbar()

In [ ]:
A,B = ngpsi.NdGaussianProcessSensitivityIndicesBase.SymbolicSaltelliIndices(1)
A

In [ ]:
A.gradient([2,3])

In [ ]:
#composedDist = processSensitivityAnalysis.wrappedFunction.KLComposedDistribution
plt.yticks([0,1,2,3,4],["Module d'Young",'Diametre','Densité','Position Force','Norme Force'])
x=plt.imshow(S, cmap='CMRmap',aspect='auto')
plt.colorbar()

Test for the variance calculus

In [ ]:
outputDim0 = output2.shape[0];print(outputDim0)
inputDim = 5 #(5 dimensions)
N = 100
print(output2)

In [ ]:
ngpsi.NdGaussianProcessSensitivityIndicesBase.computeVariance(1, 
                                                              inputDim,
                                                             N,
                                                             output2)

In [ ]:
S_fo[...,50]

In [ ]:
im2 = plt.imshow(processSensitivityAnalysis.sensitivityResults.firstOrderIndices[1])
plt.show()

In [ ]:
#im2.write_png('sensitivity_rv_KL')

In [ ]:
processSensitivityAnalysis.wrappedFunction.getKLDecompositionVarNames

In [ ]:
np.sum(S_fo, axis=0)

In [ ]:
S_f